In [1]:
import requests
import json


In [7]:
requests.delete("http://localhost:9200/_ltr/")

In [8]:
requests.put("http://localhost:9200/_ltr/")

In [10]:
feature_set = {
   "featureset": {
      "features": [
         {
            "name": "title_bm25",
            "params": [
               "keywords"
            ],
            "template": {
                     "match": {
                        "title": "{{keywords}}"
                     }
               }
         },
         {
            "name": "overview_bm25",
            "params": [
               "keywords"
            ],
            "template": {
                     "match": {
                        "overview": "{{keywords}}"
                     }
               }
         }
      ]
   },
     "validation": {
      "index": "tmdb",
      "params": {
         "keywords": "rambo"
      }
   }
}

requests.put('http://localhost:9200/_ltr/_featureset/my_feature_set', 
             json=feature_set)

In [11]:
search = {
    "query": { 
    "match": {
        "title": "First Blood"
    }}
}

resp = requests.get('http://localhost:9200/tmdb/_search', json=search).json()
print(resp['hits']['hits'][0]['_source']['title'])

In [46]:
search_with_log = {
  "query": {
    "bool": {
      "filter": [
        {
          "sltr": {
            "_name": "logged_features",
            "featureset": "my_feature_set",
            "params": {
              "keywords": "rambo"
            }
          }
        },
         {
          "terms": {
            "_id": [
              "7555","1370", "1369", "1368"
            ]
          }
        }
      ]
    }
  },
  "ext": {
    "ltr_log": {
      "log_specs": {
        "name": "ltr_features",
        "named_query": "logged_features",
        "missing_as_zero": True
      }
    }
  }
}

resp = requests.get('http://localhost:9200/tmdb/_search', json=search_with_log).json()
print(json.dumps(resp['hits']['hits'][1], indent=2))

In [43]:
model = open('data/wtf.txt').read()


create_model = {
  "model": {
     "name": "wtf_model",
     "model": {
         "type": "model/ranklib",
         "definition": model
    }
  }
}

requests.post('http://localhost:9200/_ltr/_featureset/my_feature_set/_createmodel',
              json=create_model).json()

In [44]:
search = {
  "query": {
      "sltr": {
          "params": {
              "keywords": "rambo"
          },
          "model": "wtf_model"
      }
  }
}
resp = requests.get('http://localhost:9200/tmdb/_search', json=search).json()

In [45]:
for hit in resp['hits']['hits']:
    print(hit['_source']['title'])
